### Preliminary work: Importing required packages, importing parameters setting, defining functions

In [178]:
# Required packages
import numpy as np
import pandas as pd
from scipy.io import loadmat
from scipy.stats import norm
import pdb
import warnings
import SolveLinSys1
import SolveLinSys2
import time
import sys
import scipy.interpolate
sys.stdout.flush()
# import line_profiler
# %load_ext line_profiler

In [179]:
# Data Cleansing
x = loadmat('climate_pars.mat')
for key,val in x.items():
    if key[0:2] == '__':
        pass
    elif key == 'filename' or key == 's' or key == 's0' or key =='s1' or key == 's2':
        pass
    else:
        (height, width) = val.shape
        if width == 1:             
            if height == 1:    # one number
                exec(key +'=val[0,0]')
            else:              # a column array
                exec(key +'=val[:,0]')
        else:
            if height == 1:    # a row array
                exec(key +'=val[0,:]')
            else:
                exec(key +'=val')

In [180]:
def centra_diff(data, dim, order, dlt, cap = None):  # compute the central difference derivatives for given input and dimensions
    res = np.zeros(data.shape)
    if order == 1:                    # first order derivatives
        
        if dim == 0:                  # to first dimension

            res[1:-1,:,:] = (1 / (2 * dlt)) * (data[2:,:,:] - data[:-2,:,:])
            res[-1,:,:] = (1 / dlt) * (data[-1,:,:] - data[-2,:,:])
            res[0,:,:] = (1 / dlt) * (data[1,:,:] - data[0,:,:])

        elif dim == 1:                # to second dimension

            res[:,1:-1,:] = (1 / (2 * dlt)) * (data[:,2:,:] - data[:,:-2,:])
            res[:,-1,:] = (1 / dlt) * (data[:,-1,:] - data[:,-2,:])
            res[:,0,:] = (1 / dlt) * (data[:,1,:] - data[:,0,:])

        elif dim == 2:                # to third dimension

            res[:,:,1:-1] = (1 / (2 * dlt)) * (data[:,:,2:] - data[:,:,:-2])
            res[:,:,-1] = (1 / dlt) * (data[:,:,-1] - data[:,:,-2])
            res[:,:,0] = (1 / dlt) * (data[:,:,1] - data[:,:,0])

        else:
            raise ValueError('wrong dim')
            
    elif order == 2:
        
        if dim == 0:                  # to first dimension

            res[1:-1,:,:] = (1 / dlt ** 2) * (data[2:,:,:] + data[:-2,:,:] - 2 * data[1:-1,:,:])
            res[-1,:,:] = (1 / dlt ** 2) * (data[-1,:,:] + data[-3,:,:] - 2 * data[-2,:,:])
            res[0,:,:] = (1 / dlt ** 2) * (data[2,:,:] + data[0,:,:] - 2 * data[1,:,:])

        elif dim == 1:                # to second dimension

            res[:,1:-1,:] = (1 / dlt ** 2) * (data[:,2:,:] + data[:,:-2,:] - 2 * data[:,1:-1,:])
            res[:,-1,:] = (1 / dlt ** 2) * (data[:,-1,:] + data[:,-3,:] - 2 * data[:,-2,:])
            res[:,0,:] = (1 / dlt ** 2) * (data[:,2,:] + data[:,0,:] - 2 * data[:,1,:])

        elif dim == 2:                # to third dimension

            res[:,:,1:-1] = (1 / dlt ** 2) * (data[:,:,2:] + data[:,:,:-2] - 2 * data[:,:,1:-1])
            res[:,:,-1] = (1 / dlt ** 2) * (data[:,:,-1] + data[:,:,-3] - 2 * data[:,:,-2])
            res[:,:,0] = (1 / dlt ** 2) * (data[:,:,2] + data[:,:,0] - 2 * data[:,:,1])

        else:
            raise ValueError('wrong dim')
        
    else:
        raise ValueError('wrong order')
        
    if cap is not None:
        res[res < cap] = cap
    return res

def quad_points_legendre(n):
    u = np.sqrt(1 / (4 - 1 / np.linspace(1,n-1,n-1)**2))  # upper diag
    [lambda0,V] = np.linalg.eig(np.diagflat(u,1) + np.diagflat(u,-1))  # V's column vectors are the main d
    i = np.argsort(lambda0)
    Vtop = V[0,:]
    Vtop = Vtop[i]
    w = 2 * Vtop ** 2
    return (lambda0[i],w)

def quad_points_hermite(n):
    i = np.linspace(1,n-1,n-1)
    a = np.sqrt(i / 2.0)
    [lambda0,V] = np.linalg.eig(np.diagflat(a,1) + np.diagflat(a,-1))
    i = np.argsort(lambda0)
    Vtop = V[0,:]
    Vtop = Vtop[i]
    w = np.sqrt(np.pi) * Vtop ** 2
    return (lambda0[i],w)


def quad_int(f,a,b,n,method):
    """
This function takes a function f to integrate from the multidimensional
interval specified by the row vectors a and b. N different points are used
in the quadrature method. Legendre and Hermite basis functions are
currently supported. In the case of Hermite methodology b is the normal
density and a is the normal mean.

Created by John Wilson (johnrwilson@uchicago.edu) & Updaed by Jiaming Wang (Jiamingwang@uchicago.edu)
    """
    if method == 'legendre':
        
        (xs,ws) = quad_points_legendre(n)
        g = lambda x: f((b-a) * 0.5  * x + (a + b) * 0.5)
        s = np.prod((b-a) * 0.5)                ######## Why using prod here?
        
    elif method == 'hermite':
        
        (xs,ws) = quad_points_hermite(n)
        g = lambda x: f(np.sqrt(2) * b * x + a)
        s = 1 / np.sqrt(np.pi)
        
    else:
        raise TypeError('Wrong polynomials specification')
    
    
    tp = type(a)
    if tp is np.float64 or tp is int or tp is np.double:
        res = 0
        for i in range(n):
#             pdb.set_trace()
            res += ws[i] * g(xs[i])
    else:
        raise ValueError('dimension is not 1')
    
    return s * res


def cap(x, lb, ub):
    if x <= ub or x >= lb:
        return x
    else:
        if x > ub:
            return ub
        else:
            return lb

In [205]:
np.var(par_lambda_McD, ddof=1), np.std(par_lambda_McD, ddof=1)

(2.430335570523782e-07, 0.0004929843375325206)

### Preference Damage

In [181]:
start_time = time.time()
McD = np.loadtxt('TCRE_MacDougallEtAl2017_update.txt')
par_lambda_McD = McD / 1000

beta_f = np.mean(par_lambda_McD)  # Climate sensitivity parameter, MacDougall (2017)
var_beta_f = np.var(par_lambda_McD, ddof=1)  # varaiance of climate sensitivity parameters
lambda0 = 1.0 / var_beta_f 
delta = 0.01        # subjective rate of discount
alpha = 0.032       # 
sigma_g = 0.02
sigma_k = 0.0161
sigma_r = 0.0339
A_O = 0.115000000000000
Gamma = 0.0600
Theta = 1. / 16.666666666666668
Alpha = -0.034977443912449
Gamma_r = 0.112733407891680
Theta_r = 0.142857142857143
# parameters for damage function
power = 2
gamma_1 = 0.00017675
gamma_2 = 2. * 0.0022
gamma_2_plus = 2. * 0.0197
sigma_1 = 0
sigma_2 = 0
rho_12 = 0
f_bar = 2
crit = 2
F0 = 1
sigma = np.matrix([[sigma_1 ** 2, rho_12], [rho_12, sigma_2 ** 2]])
Sigma = np.matrix([[var_beta_f, 0, 0], [0, sigma_1 ** 2, rho_12], [0, rho_12, sigma_2 ** 2]])
dee = np.matrix([gamma_1 + gamma_2 * F0 + gamma_2_plus * (F0 - f_bar) ** 2 * (F0 >= 2), beta_f, beta_f * F0])
sigma_d = float(np.sqrt(dee * Sigma * dee.T))
xi_d = -1 * (1 - alpha)

#  weight on Nordhaus
weight = 0.0
bar_gamma_2_plus = weight * 0 + (1 - weight) * gamma_2_plus

theta = 4500  #

# Time step
dt = 0.5
n = 30

# Specifying Tolerance level
tol = 1e-10

# Solving HJB
r_min = 0
r_max = 9
t_min = 0
t_max = 4000
k_min = 0
k_max = 9


nr = 30
nt = 40
nk = 25
r = np.linspace(r_min, r_max, nr)
t = np.linspace(t_min, t_max, nt)
k = np.linspace(k_min, k_max, nk)

hr = r[1] - r[0]
hk = k[1] - k[0]
ht = t[1] - t[0]

(r_mat, t_mat, k_mat) = np.meshgrid(r,t,k, indexing = 'ij')

quadrature = 'legendre'
a = beta_f - 5 * np.sqrt(var_beta_f)
b = beta_f + 5 * np.sqrt(var_beta_f)

v0 = alpha * r_mat + (1-alpha) * k_mat - beta_f * t_mat
v1_initial = v0 * np.ones(r_mat.shape)

# Applying finite difference scheme to the value function
v0_dr = centra_diff(v0,0,1,hr,1e-8) 
v0_dt = centra_diff(v0,1,1,ht)
v0_dk = centra_diff(v0,2,1,hk)

v0_drr = centra_diff(v0,0,2,hr)
v0_dtt = centra_diff(v0,1,2,ht)
v0_dkk = centra_diff(v0,2,2,hk)

B1 = v0_dr - xi_d * (gamma_1 + gamma_2 * t_mat * beta_f + gamma_2_plus * (t_mat * beta_f - f_bar) ** (power - 1) * (t_mat >= (crit / beta_f))) * beta_f * np.exp(r_mat) - v0_dt * np.exp(r_mat)
C1 = - delta * alpha
e = -C1 / B1
e_hat = e
Acoeff = np.exp(r_mat - k_mat)
Bcoeff = delta * (1-alpha) / (np.exp(-r_mat + k_mat) * v0_dr * Gamma_r * 0.5) + v0_dk * Gamma / (np.exp(-r_mat + k_mat) * v0_dr * Gamma_r * 0.5)
Ccoeff = -A_O  - Theta
f = ((-Bcoeff + np.sqrt(Bcoeff ** 2 - 4 * Acoeff * Ccoeff)) / (2 * Acoeff)) ** 2
i_k = (v0_dk * Gamma / (np.exp(-r_mat + k_mat) * v0_dr * Gamma_r * 0.5)) * (f ** 0.5) - Theta

a_1 = np.zeros(r_mat.shape)
b_1 = xi_d * e_hat * np.exp(r_mat) * gamma_1
c_1 = 2 * xi_d * e_hat * np.exp(r_mat) * t_mat * gamma_2
lambda_tilde_1 = lambda0 + c_1 * theta
beta_tilde_1 = beta_f - c_1 * theta / lambda_tilde_1 * beta_f - theta / lambda_tilde_1 * b_1
I_1 = a_1 - 0.5 * np.log(lambda0) / theta + 0.5 * np.log(lambda_tilde_1) / theta + 0.5 * lambda0 * beta_f ** 2 / theta - 0.5 * lambda_tilde_1 * (beta_tilde_1) ** 2 / theta
#     R_1 = theta.*(I_1-(a_1+b_1.*beta_tilde_1+c_1./2.*(beta_tilde_1).^2+c_1./2./lambda_tilde_1));
R_1 = theta * (I_1 - (a_1 + b_1 * beta_tilde_1 + c_1 * beta_tilde_1 ** 2 + c_1 / lambda_tilde_1))
J_1_without_e = xi_d * (gamma_1 * beta_tilde_1 + gamma_2 * t_mat * (beta_tilde_1 ** 2 + 1 / lambda_tilde_1)) * np.exp(r_mat)

pi_tilde_1 = weight * np.exp(-theta * I_1)
r_multi = np.exp(r)
r_multi = r_multi[:,np.newaxis,np.newaxis]

scale_2_fnc = lambda x: np.exp(-theta * xi_d * (gamma_1 * x + gamma_2 * x ** 2 * t_mat + gamma_2_plus * x * (x * t_mat - f_bar) ** (power - 1) * ((x * t_mat - f_bar) >= 0)) * np.exp(r_mat) * e_hat)  * norm.pdf(x,beta_f,np.sqrt(var_beta_f))
scale_2 = quad_int(scale_2_fnc, a, b, n, 'legendre')

q2_tilde_fnc = lambda x: np.exp(-theta * xi_d * (gamma_1 * x + gamma_2 * x ** 2 * t_mat + gamma_2_plus * x * (x * t_mat - f_bar) ** (power - 1) * ((x * t_mat - f_bar) >= 0)) * np.exp(r_mat) * e_hat) / scale_2
I_2 = -1 / theta * np.log(scale_2)

J_2_without_e_fnc = lambda x: xi_d * np.exp(r_mat) * q2_tilde_fnc(x) * (gamma_1 * x + gamma_2 * t_mat * x ** 2 + gamma_2_plus * x * (x * t_mat - f_bar) ** (power - 1) * ((x * t_mat - f_bar) >= 0)) * norm.pdf(x,beta_f,np.sqrt(var_beta_f))
J_2_without_e = quad_int(J_2_without_e_fnc, a, b, n, 'legendre')
J_2_with_e = J_2_without_e * e_hat

R_2 = theta * (I_2 - J_2_with_e)
pi_tilde_2 = (1 - weight) * np.exp(-theta * I_2)
pi_tilde_1_norm = pi_tilde_1 / (pi_tilde_1 + pi_tilde_2)
pi_tilde_2_norm = 1 - pi_tilde_1_norm

expec_e_sum = (pi_tilde_1_norm * J_1_without_e + pi_tilde_2_norm * J_2_without_e)

B1 = v0_dr - v0_dt * np.exp(r_mat) - expec_e_sum
C1 = -delta * alpha
e = -C1 / B1
e_star = e

J_1 = J_1_without_e * e_star
J_2 = J_2_without_e * e_star

I_term = -1 / theta * np.log(pi_tilde_1 + pi_tilde_2)

R_1 = theta * (I_1 - J_1)
R_2 = theta * (I_2 - J_2)
drift_distort = (pi_tilde_1_norm * J_1 + pi_tilde_2_norm * J_2)

if weight == 0 or weight == 1:
    RE = pi_tilde_1_norm * R_1 + pi_tilde_2_norm * R_2
else:
    RE = pi_tilde_1_norm * R_1 + pi_tilde_2_norm * R_2 + pi_tilde_1_norm * np.log(
        pi_tilde_1_norm / weight) + pi_tilde_2_norm * np.log(pi_tilde_2_norm / (1 - weight))

RE_total = 1 / theta * RE

### PDE Inputs

A_ = -delta * np.ones(r_mat.shape)
B_r = -e_star + Gamma_r * (f ** Theta_r) - 0.5 * (sigma_r ** 2)
B_k = Alpha + Gamma * np.log(1 + i_k / Theta) - 0.5 * (sigma_k ** 2)
B_t = e_star * np.exp(r_mat)
C_rr = 0.5 * sigma_r ** 2 * np.ones(r_mat.shape)
C_kk = 0.5 * sigma_k ** 2 * np.ones(r_mat.shape)
C_tt = np.zeros(r_mat.shape)

D_ = delta * alpha * np.log(e_star) + delta * alpha * r_mat + delta * (1 - alpha) * (np.log(A_O - i_k - f * np.exp(r_mat - k_mat)) + k_mat) + I_term #  + drift_distort + RE_total

stateSpace = np.hstack([r_mat.reshape(-1,1,order = 'F'),t_mat.reshape(-1,1,order = 'F'),k_mat.reshape(-1,1,order = 'F')])
A = A_.reshape(-1,1,order = 'F')
B = np.hstack([B_r.reshape(-1,1,order = 'F'),B_t.reshape(-1,1,order = 'F'),B_k.reshape(-1,1,order = 'F')])
C = np.hstack([C_rr.reshape(-1,1,order = 'F'), C_tt.reshape(-1,1,order = 'F'), C_kk.reshape(-1,1,order = 'F')])
D = D_.reshape(-1,1,order = 'F')
v01 = v0.reshape(-1,1,order = 'F')
dt = dt

out = SolveLinSys1.solvels(stateSpace, A,B,C,D,v01,dt)
out_comp = out[2].reshape(v0.shape,order = "F")
episode = 0
PDE_rhs = A_ * v0 + B_r * v0_dr + B_t * v0_dt + B_k * v0_dk + C_rr * v0_drr + C_kk * v0_dkk + C_tt * v0_dtt + D_
PDE_Err = np.max(abs(PDE_rhs))
FC_Err = np.max(abs((out_comp - v0)))
print("Episode %d: PDE Error: %f; False Transient Error: %f; Iterations: %d; CG Error: %f" %(episode, PDE_Err, FC_Err, out[0], out[1]))

v0 = out_comp
vold = v1_initial

while np.max(abs((out_comp - vold) / dt)) > tol:
    vold = v0.copy()
    v0_dr = centra_diff(v0, 0, 1, hr, 1e-8)
    v0_dt = centra_diff(v0, 1, 1, ht)
    v0_dk = centra_diff(v0, 2, 1, hk)

    v0_drr = centra_diff(v0, 0, 2, hr)
    v0_dtt = centra_diff(v0, 1, 2, ht)
    v0_dkk = centra_diff(v0, 2, 2, hk)
    e_hat = e_star

    f = ((A_O + Theta) * np.exp(-r_mat + k_mat) * (v0_dr * Gamma_r * Theta_r) / ((v0_dr * Gamma_r * Theta_r) * f ** (Theta_r) + (delta * (1-alpha) + v0_dk * Gamma))) ** (1 / (1 - Theta_r))
    f = f * (v0_dr > 1e-8)
    i_k = ((v0_dk * Gamma / (np.exp(-r_mat + k_mat) * v0_dr * Gamma_r * Theta_r)) * (f ** (1 - Theta_r)) - Theta) * (v0_dr > 1e-8) + (v0_dr <= 1e-8) * (v0_dk * Gamma * A_O - delta * (1-alpha) * Theta) / (delta * (1-alpha) + v0_dk * Gamma)

    a_1 = np.zeros(r_mat.shape);
    b_1 = xi_d * e_hat * np.exp(r_mat) * gamma_1
    c_1 = 2 * xi_d * e_hat * np.exp(r_mat) * t_mat * gamma_2
    lambda_tilde_1 = lambda0 + c_1 * theta

    beta_tilde_1 = beta_f - c_1 * theta / lambda_tilde_1 * beta_f - theta / lambda_tilde_1 * b_1

    I_1 = a_1 - 0.5 * np.log(lambda0) / theta + 0.5 * np.log(lambda_tilde_1) / theta + 0.5 * lambda0 * beta_f ** 2 / theta - 0.5 * lambda_tilde_1 * (beta_tilde_1) ** 2 / theta

    R_1 = theta * (I_1 - (a_1 + b_1 * beta_tilde_1 + c_1 * beta_tilde_1 ** 2 + c_1 / lambda_tilde_1))
    J_1_without_e = xi_d * (gamma_1 * beta_tilde_1 + gamma_2 * t_mat * (beta_tilde_1 ** 2 + 1 / lambda_tilde_1)) * np.exp(r_mat)
    pi_tilde_1 = weight * np.exp(-theta * I_1)

    scale_2_fnc = lambda x: np.exp(-theta * xi_d * (
                gamma_1 * x + gamma_2 * x ** 2 * t_mat + gamma_2_plus * x * (x * t_mat - f_bar) ** (power - 1) * (
                    (x * t_mat - f_bar) >= 0)) * np.exp(r_mat) * e_hat) * norm.pdf(x, beta_f, np.sqrt(var_beta_f))
    scale_2 = quad_int(scale_2_fnc, a, b, n, 'legendre')

    q2_tilde_fnc = lambda x: np.exp(-theta * xi_d * (
                gamma_1 * x + gamma_2 * x ** 2 * t_mat + gamma_2_plus * x * (x * t_mat - f_bar) ** (power - 1) * (
                    (x * t_mat - f_bar) >= 0)) * np.exp(r_mat) * e_hat) / scale_2
    I_2 = -1 / theta * np.log(scale_2)

    J_2_without_e_fnc = lambda x: xi_d * np.exp(r_mat) * q2_tilde_fnc(x) * (
                gamma_1 * x + gamma_2 * t_mat * x ** 2 + gamma_2_plus * x * (x * t_mat - f_bar) ** (power - 1) * (
                    (x * t_mat - f_bar) >= 0)) * norm.pdf(x, beta_f, np.sqrt(var_beta_f))
    J_2_without_e = quad_int(J_2_without_e_fnc, a, b, n, 'legendre')

    J_2_with_e = J_2_without_e * e_hat
    R_2 = theta * (I_2 - J_2_with_e)
    pi_tilde_2 = (1 - weight) * np.exp(-theta * I_2)
    pi_tilde_1_norm = pi_tilde_1 / (pi_tilde_1 + pi_tilde_2)
    pi_tilde_2_norm = 1 - pi_tilde_1_norm

    expec_e_sum = (pi_tilde_1_norm * J_1_without_e + pi_tilde_2_norm * J_2_without_e)

    B1 = v0_dr - v0_dt * np.exp(r_mat) - expec_e_sum
    C1 = -delta * alpha
    e = -C1 / B1
    e_star = e

    J_1 = J_1_without_e * e_star
    J_2 = J_2_without_e * e_star
    I_term = -1 / theta * np.log(pi_tilde_1 + pi_tilde_2)
    R_1 = theta * (I_1 - J_1)
    R_2 = theta * (I_2 - J_2)
    drift_distort = (pi_tilde_1_norm * J_1 + pi_tilde_2_norm * J_2)

    if weight == 0 or weight == 1:
        RE = pi_tilde_1_norm * R_1 + pi_tilde_2_norm * R_2
    else:
        RE = pi_tilde_1_norm * R_1 + pi_tilde_2_norm * R_2 + pi_tilde_1_norm * np.log(
            pi_tilde_1_norm / weight) + pi_tilde_2_norm * np.log(pi_tilde_2_norm / (1 - weight))

    RE_total = 1 / theta * RE

    A_ = -delta * np.ones(r_mat.shape)
    B_r = -e_star + Gamma_r * (f ** Theta_r) - 0.5 * (sigma_r ** 2)
    B_k = Alpha + Gamma * np.log(1 + i_k / Theta) - 0.5 * (sigma_k ** 2)
    B_t = e_star * np.exp(r_mat)
    C_rr = 0.5 * sigma_r ** 2 * np.ones(r_mat.shape)
    C_kk = 0.5 * sigma_k ** 2 * np.ones(r_mat.shape)
    C_tt = np.zeros(r_mat.shape)

    D_ = delta * alpha * np.log(e_star) + delta * alpha * r_mat + delta * (1 - alpha) * (
                np.log(A_O - i_k - f * np.exp(r_mat - k_mat)) + k_mat) + I_term  # + drift_distort + RE_total

    stateSpace = np.hstack(
        [r_mat.reshape(-1, 1, order='F'), t_mat.reshape(-1, 1, order='F'), k_mat.reshape(-1, 1, order='F')])
    A = A_.reshape(-1, 1, order='F')
    B = np.hstack([B_r.reshape(-1, 1, order='F'), B_t.reshape(-1, 1, order='F'), B_k.reshape(-1, 1, order='F')])
    C = np.hstack([C_rr.reshape(-1, 1, order='F'), C_tt.reshape(-1, 1, order='F'), C_kk.reshape(-1, 1, order='F')])
    D = D_.reshape(-1, 1, order='F')
    v01 = v0.reshape(-1, 1, order='F')
    dt = dt

    out = SolveLinSys1.solvels(stateSpace, A,B,C,D,v01,dt)
    out_comp = out[2].reshape(v0.shape,order = "F")
    PDE_rhs = A_ * v0 + B_r * v0_dr + B_t * v0_dt + B_k * v0_dk + C_rr * v0_drr + C_kk * v0_dkk + C_tt * v0_dtt + D_
    PDE_Err = np.max(abs(PDE_rhs))
    FC_Err = np.max(abs((out_comp - v0)))

    #     mdl = SolveLinSys1.model(stateSpace, A,B,C,D,v01,dt)
    #     mdl.solvemodel()
    #     out = mdl.getresult()
    #     out_comp = out.reshape(v0.shape,order = "F")
    episode += 1

    v0 = out_comp
    if episode % 100 == 0:
        print("Episode %d: PDE Error: %f; False Transient Error: %f; Iterations: %d; CG Error: %f" %(episode, PDE_Err, FC_Err, out[0], out[1]))

print("Episode %d: PDE Error: %f" % (episode, np.max((out_comp - vold) / dt)))
print("--- %s seconds ---" % (time.time() - start_time))

Episode 0: PDE Error: 0.053022; False Transient Error: 0.026376; Iterations: 11; CG Error: 0.000000
Episode 100: PDE Error: 0.031987; False Transient Error: 0.015908; Iterations: 8; CG Error: 0.000000
Episode 200: PDE Error: 0.019133; False Transient Error: 0.009519; Iterations: 7; CG Error: 0.000000
Episode 300: PDE Error: 0.011584; False Transient Error: 0.005763; Iterations: 6; CG Error: 0.000000
Episode 400: PDE Error: 0.007020; False Transient Error: 0.003493; Iterations: 6; CG Error: 0.000000
Episode 500: PDE Error: 0.004260; False Transient Error: 0.002118; Iterations: 6; CG Error: 0.000000
Episode 600: PDE Error: 0.002592; False Transient Error: 0.001289; Iterations: 5; CG Error: 0.000000
Episode 700: PDE Error: 0.001579; False Transient Error: 0.000785; Iterations: 5; CG Error: 0.000000
Episode 800: PDE Error: 0.000962; False Transient Error: 0.000478; Iterations: 5; CG Error: 0.000000
Episode 900: PDE Error: 0.000587; False Transient Error: 0.000291; Iterations: 5; CG Error: 

In [ ]:
αₜ

### Jupyter noebook solves the model in 1021 seconds (3096 iterations) while MATLAB took about 200 seconds. The differenceS of solved result and parameters are tiny as shown below.

In [44]:
for_check = loadmat('HJB_level_result.mat')
np.max(abs(for_check['out_comp'] - out_comp))

ValueError: operands could not be broadcast together with shapes (30,30,25) (30,40,25) 

### Simulation

In [182]:
T = 100
pers = 4 * T
dt = T / pers
nDims = 5
its = 1

gridpoints = (r, t, k)
e_func_r = scipy.interpolate.RegularGridInterpolator(gridpoints, e)
e_func = lambda x: e_func_r([np.log(x[0]), x[2], np.log(x[1])])[0]
f_func_r = scipy.interpolate.RegularGridInterpolator(gridpoints, f)
f_func = lambda x: max(f_func_r([np.log(x[0]), x[2], np.log(x[1])])[0], 0)
i_kfunc_r = scipy.interpolate.RegularGridInterpolator(gridpoints, i_k)
i_kfunc = lambda x: i_kfunc_r([np.log(x[0]), x[2], np.log(x[1])])[0]
v_drfunc_r = scipy.interpolate.RegularGridInterpolator(gridpoints, v0_dr)
v_drfunc = lambda x: v_drfunc_r([np.log(x[0]), x[2], np.log(x[1])])[0]
v_dtfunc_r = scipy.interpolate.RegularGridInterpolator(gridpoints, v0_dt)
v_dtfunc = lambda x: v_dtfunc_r([np.log(x[0]), x[2], np.log(x[1])])[0]
v_dkfunc_r = scipy.interpolate.RegularGridInterpolator(gridpoints, v0_dk)
v_dkfunc = lambda x: v_dkfunc_r([np.log(x[0]), x[2], np.log(x[1])])[0]
v_func_r = scipy.interpolate.RegularGridInterpolator(gridpoints, out_comp)
v_func = lambda x: v_func_r([np.log(x[0]), x[2], np.log(x[1])])[0]


pi_tilde_1_func_r = scipy.interpolate.RegularGridInterpolator(gridpoints, pi_tilde_1)
pi_tilde_1_func = lambda x: pi_tilde_1_func_r([np.log(x[0]), x[2], np.log(x[1])])[0]
pi_tilde_2_func_r = scipy.interpolate.RegularGridInterpolator(gridpoints, pi_tilde_2)
pi_tilde_2_func = lambda x: pi_tilde_2_func_r([np.log(x[0]), x[2], np.log(x[1])])[0]

bar_gamma_2_plus = (1-weight) * gamma_2_plus
base_model_drift_func = lambda x: np.exp(r_mat) * e * (gamma_1 * x + gamma_2 * x ** 2 * t_mat + bar_gamma_2_plus * x * (x * t_mat - f_bar) ** (power - 1) * ((x * t_mat - f_bar) >= 0)) * norm.pdf(x,beta_f,np.sqrt(var_beta_f))
base_model_drift =  quad_int(base_model_drift_func, a, b, n, 'legendre')

mean_nordhaus = beta_tilde_1
lambda_tilde_nordhaus = lambda_tilde_1
nordhaus_model_drift = (gamma_1 * mean_nordhaus + gamma_2 * (1 / lambda_tilde_nordhaus + mean_nordhaus * 2) * t_mat) * np.exp(r_mat) * e

weitzman_model_drift_func = lambda x: np.exp(r_mat) * e * q2_tilde_fnc(x) * (gamma_1 * x + gamma_2 * x ** 2 * t_mat + bar_gamma_2_plus * x * (x * t_mat - f_bar) ** (power - 1) * ((x * t_mat - f_bar) >= 0)) * norm.pdf(x,beta_f,np.sqrt(var_beta_f))
weitzman_model_drift = quad_int(weitzman_model_drift_func, a, b, n, 'legendre')

nordhaus_drift_func_r = scipy.interpolate.RegularGridInterpolator(gridpoints, nordhaus_model_drift)
nordhaus_drift_func = lambda x: nordhaus_drift_func_r([np.log(x[0]), x[2], np.log(x[1])])[0]

weitzman_drift_func_r = scipy.interpolate.RegularGridInterpolator(gridpoints, weitzman_model_drift)
weitzman_drift_func = lambda x: weitzman_drift_func_r([np.log(x[0]), x[2], np.log(x[1])])[0]

base_drift_func_r = scipy.interpolate.RegularGridInterpolator(gridpoints, base_model_drift)
base_drift_func = lambda x: base_drift_func_r([np.log(x[0]), x[2], np.log(x[1])])[0]

# function handles
muR = lambda x: -e_func(x) + Gamma_r * f_func(x) ** Theta_r
muK = lambda x: (Alpha + Gamma * np.log(1 + i_kfunc(x) / Theta))
muT = lambda x: e_func(x) * x[0]
muD_base = lambda x: base_drift_func(x)
muD_tilted = lambda x: pi_tilde_1_func(x) * nordhaus_drift_func(x) + (1 - pi_tilde_1_func(x)) * weitzman_drift_func(x)

sigmaR = lambda x: np.zeros(x[:5].shape)
sigmaK = lambda x: np.zeros(x[:5].shape)
sigmaT = lambda x: np.zeros(x[:5].shape)
sigmaD = lambda x: np.zeros(x[:5].shape)

# initial points
R_0 = 650
K_0 = 80 / A_O
T_0 = 870 - 580
initial_val = np.array([R_0, K_0, T_0])
D_0_base = muD_base(initial_val)
D_0_tilted = muD_tilted(initial_val)

# Set bounds
R_max = np.exp(r_max)
K_max = np.exp(k_max)
T_max = t_max
D_max = 5.0

R_min = np.exp(r_min)
K_min = np.exp(k_min)
T_min = t_min
D_min = -5

upperbounds = np.array([R_max, K_max, T_max, D_max, D_max])
lowerbounds = np.array([R_min, K_min, T_min, D_min, D_min])

hists = np.zeros([pers, nDims, its])
hists2 = hists.copy()
e_hists = np.zeros([pers,its])
e_hists2 = e_hists.copy()
f_hists = np.zeros([pers,its])
f_hists2 = f_hists.copy()
i_k_hists = np.zeros([pers,its])
i_k_hists2 = i_k_hists.copy()

v_dr_hists2 = np.zeros([pers,its])
v_dt_hists2 = np.zeros([pers,its])
v_dk_hists2 = np.zeros([pers,its])
v_hists2 = np.zeros([pers,its])

for iters in range(0,its):
    hist2 = np.zeros([pers,nDims])
    e_hist2 = np.zeros([pers,1])
    i_k_hist2 = np.zeros([pers,1])
    f_hist2 = np.zeros([pers,1])
    
    v_dr_hist2 = np.zeros([pers,1])
    v_dt_hist2 = np.zeros([pers,1])
    v_dk_hist2 = np.zeros([pers,1])
    v_hist2 = np.zeros([pers,1])
    
    hist2[0,:] = [R_0, K_0, T_0, D_0_base, D_0_tilted]
    e_hist2[0] = e_func(hist2[0,:]) * hist2[0,0]
    i_k_hist2[0] = i_kfunc(hist2[0,:]) * hist2[0,1]
    f_hist2[0] = f_func(hist2[0,:]) * hist2[0,0]
    v_dr_hist2[0] = v_drfunc(hist2[0,:])
    v_dt_hist2[0] = v_dtfunc(hist2[0,:])
    v_dk_hist2[0] = v_dkfunc(hist2[0,:])
    v_hist2[0] = v_func(hist2[0,:])
    
    for j in range(1,pers):
        shock = norm.rvs(0,np.sqrt(dt),nDims)
        hist2[j,0] = cap(hist2[j-1,0] * np.exp((muR(hist2[j-1,:])- 0.5 * sum((sigmaR(hist2[j-1,:])) ** 2))* dt + sigmaR(hist2[j-1,:]).dot(shock)),lowerbounds[0], upperbounds[0])
        hist2[j,1] = cap(hist2[j-1,1] * np.exp((muK(hist2[j-1,:])- 0.5 * sum((sigmaK(hist2[j-1,:])) ** 2))* dt + sigmaK(hist2[j-1,:]).dot(shock)),lowerbounds[1], upperbounds[1])
        hist2[j,2] = cap(hist2[j-1,2] + muT(hist2[j-1,:]) * dt + sigmaT(hist2[j-1,:]).dot(shock), lowerbounds[2], upperbounds[2])
        hist2[j,3] = cap(hist2[j-1,3] + muD_base(hist2[j-1,:]) * dt + sigmaD(hist2[j-1,:]).dot(shock), lowerbounds[3], upperbounds[3])
        hist2[j,4] = cap(hist2[j-1,4] + muD_tilted(hist2[j-1,:]) * dt + sigmaD(hist2[j-1,:]).dot(shock), lowerbounds[4], upperbounds[4])
        
        e_hist2[j] = e_func(hist2[j-1,:]) * hist2[j-1,0]
        i_k_hist2[j] = i_kfunc(hist2[j-1,:]) * hist2[j-1,1]
        f_hist2[j] = f_func(hist2[j-1,:]) * hist2[j-1,0]
        
        v_dr_hist2[j] = v_drfunc(hist2[j-1,:])
        v_dt_hist2[j] = v_dtfunc(hist2[j-1,:])
        v_dk_hist2[j] = v_dkfunc(hist2[j-1,:])
        v_hist2[j] = v_func(hist2[j-1,:])
        
    hists2[:,:,iters] = hist2
    e_hists2[:,[iters]] = e_hist2
    i_k_hists2[:,[iters]] = i_k_hist2
    f_hists2[:,[iters]] = f_hist2
    
    v_dr_hists2[:,[iters]] = v_dr_hist2
    v_dt_hists2[:,[iters]] = v_dt_hist2
    v_dk_hists2[:,[iters]] = v_dk_hist2
    v_hists2[:,[iters]] = v_hist2

### SCC Calculation Feyman Kac

In [183]:
# Base model
base_model_flow_func = lambda x: (gamma_2 * x ** 2 + bar_gamma_2_plus * x ** 2 * ((x * t_mat - f_bar) >=0)) * np.exp(r_mat) * e *  norm.pdf(x,beta_f,np.sqrt(var_beta_f))
base_model_flow = quad_int(base_model_flow_func, a, b, n, 'legendre')
flow_base = base_model_flow

# input for solver

A = -delta * np.ones(r_mat.shape)
B_r = -e_star + Gamma_r * (f ** Theta_r) - 0.5 * (sigma_r ** 2)
B_k = Alpha + Gamma * np.log(1 + i_k / Theta) - 0.5 * (sigma_k ** 2)
B_t = e_star * np.exp(r_mat)
C_rr = 0.5 * sigma_r ** 2 * np.ones(r_mat.shape)
C_kk = 0.5 * sigma_k ** 2 * np.ones(r_mat.shape)
C_tt = np.zeros(r_mat.shape)

D = flow_base
stateSpace = np.hstack([r_mat.reshape(-1, 1, order='F'), t_mat.reshape(-1, 1, order='F'), k_mat.reshape(-1, 1, order='F')])
A = A.reshape(-1, 1, order='F')
B = np.hstack([B_r.reshape(-1, 1, order='F'), B_t.reshape(-1, 1, order='F'), B_k.reshape(-1, 1, order='F')])
C = np.hstack([C_rr.reshape(-1, 1, order='F'), C_tt.reshape(-1, 1, order='F'), C_kk.reshape(-1, 1, order='F')])
D = D.reshape(-1, 1, order='F')
v01 = v0.reshape(-1, 1, order='F') * 0
dt = 1.0

out = SolveLinSys2.solvels(stateSpace, A, B, C, D, v01, dt)
v0_base = out[2].reshape(v0.shape, order="F")

# Worst condition
mean_nordhaus = beta_tilde_1
lambda_tilde_nordhause = lambda_tilde_1

scale_2_1_fnc = lambda x: np.exp(-theta * xi_d * (
                gamma_1 * x + gamma_2 * x ** 2 * t_mat + gamma_2_plus * x * (x * t_mat - f_bar) ** (power - 1) * (
                    (x * t_mat - f_bar) >= 0)) * np.exp(r_mat) * e) * norm.pdf(x, beta_f, np.sqrt(var_beta_f))
scale_2_1 = quad_int(scale_2_fnc, a, b, n, 'legendre')

q2_tilde_1_fnc = lambda x: np.exp(-theta * xi_d * (
            gamma_1 * x + gamma_2 * x ** 2 * t_mat + gamma_2_plus * x * (x * t_mat - f_bar) ** (power - 1) * (
                (x * t_mat - f_bar) >= 0)) * np.exp(r_mat) * e) / scale_2

nordhaus_model_flow = (gamma_2 * (1 / lambda_tilde_nordhaus + mean_nordhaus ** 2)) * np.exp(r_mat) * e 

weitzman_model_flow_func = lambda x: q2_tilde_1_fnc(x) * (gamma_2 * x ** 2 + gamma_2_plus * x ** 2 * ((x * t_mat - f_bar) >= 0 ) * np.exp(r_mat) * e * norm.pdf(x,beta_f,np.sqrt(var_beta_f)))
weitzman_model_flow = quad_int(weitzman_model_flow_func, a, b, n, 'legendre')


I_1 =  - 0.5 * np.log(lambda0) / theta + 0.5 * np.log(lambda_tilde_1) / theta + 0.5 * lambda0 * beta_f ** 2 / theta - 0.5 * lambda_tilde_1 * (beta_tilde_1) ** 2 / theta
I_2 = -1 / theta * np.log(scale_2_1)
pi_tilde_1 = weight * np.exp(-theta * I_1)
pi_tilde_2 = (1 - weight) * np.exp(-theta * I_2)
pi_tilde_1_norm = pi_tilde_1 / (pi_tilde_1 + pi_tilde_2)
pi_tilde_2_norm = 1 - pi_tilde_1_norm

flow_tilted = pi_tilde_1_norm * nordhaus_model_flow + pi_tilde_2_norm * weitzman_model_flow

A = -delta * np.ones(r_mat.shape)
B_r = -e + Gamma_r * (f ** Theta_r) - 0.5 * (sigma_r ** 2)
B_k = Alpha + Gamma * np.log(1 + i_k / Theta) - 0.5 * (sigma_k ** 2)
B_t = e * np.exp(r_mat)
C_rr = 0.5 * sigma_r ** 2 * np.ones(r_mat.shape)
C_kk = 0.5 * sigma_k ** 2 * np.ones(r_mat.shape)
C_tt = np.zeros(r_mat.shape)

D = flow_tilted

stateSpace = np.hstack([r_mat.reshape(-1, 1, order='F'), t_mat.reshape(-1, 1, order='F'), k_mat.reshape(-1, 1, order='F')])
A = A.reshape(-1, 1, order='F')
B = np.hstack([B_r.reshape(-1, 1, order='F'), B_t.reshape(-1, 1, order='F'), B_k.reshape(-1, 1, order='F')])
C = np.hstack([C_rr.reshape(-1, 1, order='F'), C_tt.reshape(-1, 1, order='F'), C_kk.reshape(-1, 1, order='F')])
D = D.reshape(-1, 1, order='F')
v01 = v0.reshape(-1, 1, order='F') * 0
dt = 1.0

out = SolveLinSys2.solvels(stateSpace, A, B, C, D, v01, dt)
v0_worst = out[2].reshape(v0.shape, order="F")



In [158]:
for_check = loadmat('SCC_worst.mat')
mdl = for_check['model']
np.max(abs(for_check['v0'] - v0_worst))

1.030989990866122e-05

In [48]:
SCC[20,2,15],SCC[21,2,15],SCC[20,3,15],SCC[20,2,16]

(22.626940327326302, 22.19557868194674, 24.498272617471095, 32.760826262972145)

In [184]:
MC = delta * (1-alpha) / (A_O * np.exp(k_mat) - i_k * np.exp(k_mat) - f * np.exp(r_mat))
ME = delta * alpha / (e * np.exp(r_mat))
SCC = 1000 * ME / MC
SCC_func_r = scipy.interpolate.RegularGridInterpolator(gridpoints, SCC)
SCC_func = lambda x: SCC_func_r([np.log(x[0]), x[2], np.log(x[1])])[0]

ME1 = v0_dr * np.exp(-r_mat)
SCC1 = 1000 * ME1 / MC
SCC1_func_r = scipy.interpolate.RegularGridInterpolator(gridpoints, SCC1)
SCC1_func = lambda x: SCC1_func_r([np.log(x[0]), x[2], np.log(x[1])])[0]

ME2_base = (1-alpha) * v0_base
SCC2_base = 1000 * ME2_base / MC
SCC2_base_func_r = scipy.interpolate.RegularGridInterpolator(gridpoints, SCC2_base)
SCC2_base_func = lambda x: SCC2_base_func_r([np.log(x[0]), x[2], np.log(x[1])])[0]

ME2b = -V_d_baseline
SCC2_V_d_baseline = 1000 * ME2b / MC
SCC2_V_d_baseline_func_r = scipy.interpolate.RegularGridInterpolator(gridpoints, SCC2_V_d_baseline)
SCC2_V_d_baseline_func = lambda x: SCC2_V_d_baseline_func_r([np.log(x[0]), x[2], np.log(x[1])])[0]

ME2_tilt = (1-alpha) * v0_worst
SCC2_tilt = 1000 * ME2_tilt / MC
SCC2_tilt_func_r = scipy.interpolate.RegularGridInterpolator(gridpoints, SCC2_tilt)
SCC2_tilt_func = lambda x: SCC2_tilt_func_r([np.log(x[0]), x[2], np.log(x[1])])[0]


ME2b = -expec_e_sum * np.exp(-r_mat)
SCC2_V_d_tilt = 1000 * ME2b / MC
SCC2_V_d_tilt_func_r = scipy.interpolate.RegularGridInterpolator(gridpoints, SCC2_V_d_tilt)
SCC2_V_d_tilt_func = lambda x: SCC2_V_d_tilt_func_r([np.log(x[0]), x[2], np.log(x[1])])[0]

SCC_values = np.zeros([pers,its])
SCC1_values = np.zeros([pers,its])
SCC2_base_values = np.zeros([pers,its])
SCC2_tilt_values = np.zeros([pers,its])
SCC2_V_d_baseline_values = np.zeros([pers,its])
SCC2_V_d_tilt_values = np.zeros([pers,its])

for tm in range(pers):
    for path in range(its):   # path is its?
        SCC_values[tm, path] = SCC_func(hists2[tm,:,path])
        SCC1_values[tm, path] = SCC1_func(hists2[tm,:,path])
        SCC2_base_values[tm, path] = SCC2_base_func(hists2[tm,:,path]) 
        SCC2_tilt_values[tm, path] = SCC2_tilt_func(hists2[tm,:,path])
        SCC2_V_d_baseline_values[tm, path] = SCC2_V_d_baseline_func(hists2[tm,:,path])
        SCC2_V_d_tilt_values[tm, path] = SCC2_V_d_tilt_func(hists2[tm,:,path])
        
SCC_total = np.mean(SCC_values,axis = 1)
SCC_private = np.mean(SCC1_values,axis = 1)
SCC2_FK_base = np.mean(SCC2_base_values,axis = 1)
SCC2_FK_tilt = np.mean(SCC2_tilt_values,axis = 1)
SCC2_V_d_baseline = np.mean(SCC2_V_d_baseline_values,axis = 1)
SCC2_V_d_tilt = np.mean(SCC2_V_d_tilt_values,axis = 1)

SCC = SCC_total
SCC1 = SCC_private
SCC2 = SCC2_FK_base + SCC2_V_d_baseline
SCC3 = SCC2_V_d_tilt - SCC2_V_d_baseline + SCC2_FK_tilt - SCC2_FK_base

print("finished")

finished


In [213]:
a_5std = a
b_5std = b
a_10std = beta_f - 10 * np.sqrt(var_beta_f)
b_10std = beta_f + 10 * np.sqrt(var_beta_f)

RE_2_func_r = scipy.interpolate.RegularGridInterpolator(gridpoints, R_2)
RE_2_func = lambda x: RE_2_func_r([np.log(x[0]), x[2], np.log(x[1])])[0]
# pi_tilde_1_func, e_func were already set up
lambda_tilde_1_func_r = scipy.interpolate.RegularGridInterpolator(gridpoints, lambda_tilde_1)
lambda_tilde_1_func = lambda x: lambda_tilde_1_func_r([np.log(x[0]), x[2], np.log(x[1])])[0]

beta_tilde_1_r = scipy.interpolate.RegularGridInterpolator(gridpoints, beta_tilde_1)
beta_tilde_1_func = lambda x: beta_tilde_1_r([np.log(x[0]), x[2], np.log(x[1])])[0]

RE_2_plot = np.zeros(pers)
weight_plot = np.zeros(pers)

for tm in range(pers):
    RE_2_plot[tm] = RE_2_func(hists2[tm,:,0])
    weight_plot[tm] = pi_tilde_1_func(hists2[tm,:,0])
beta_f_space = np.linspace(a,b,200)

# probabilities
# YEAR 0   R, K, F 
SCCs = {}

for tm in [1,100,200,300,400]:
    R0 = hists2[tm-1,0,0]
    K0 = hists2[tm-1,1,0]
    F0 = hists2[tm-1,2,0]


    scale_2_fnc_prob = lambda x: np.exp(-theta * xi_d * (
                    gamma_1 * x + gamma_2 * x ** 2 * F0 + gamma_2_plus * x * 
                    (x * F0 - f_bar) ** (power - 1) * (
                        (x * F0 - f_bar) >= 0)) * R0 * e_func([np.log(R0), F0, np.log(K0)])) * norm.pdf(x, beta_f, np.sqrt(var_beta_f))
    scale_2_prob = quad_int(scale_2_fnc_prob, a_10std, b_10std, n, 'legendre')

    q2_tilde_fnc_prob = np.exp(-theta * xi_d * (
                gamma_1 * beta_f_space + gamma_2 * beta_f_space ** 2 * F0 
                + gamma_2_plus * beta_f_space * (beta_f_space * F0 - f_bar) ** (power - 1) * (
                    (beta_f_space * F0 - f_bar) >= 0)) * R0* e_func([np.log(R0), F0, np.log(K0)])) / scale_2_prob * norm.pdf(beta_f_space,beta_f,np.sqrt(var_beta_f))
    original_dist = norm.pdf(beta_f_space,beta_f,np.sqrt(var_beta_f))
    weitzman = q2_tilde_fnc_prob

    SCCs['original_year' + str(int((tm) / 4))] = original_dist
    SCCs['weitzman_year' + str(int((tm) / 4))] = weitzman

In [206]:
testdict = {}
test_str = 'aba' + str(5)
testdict[test_str] = 10

In [214]:
pd.DataFrame(SCCs)

,original_year0,weitzman_year0,original_year25,weitzman_year25,original_year50,weitzman_year50,original_year75,weitzman_year75,original_year100,weitzman_year100
0,0.003016,0.002373,0.003016,0.001981,0.003016,0.001440,0.003016,0.000422,0.003016,0.000399
1,0.003872,0.003039,0.003872,0.002532,0.003872,0.001838,0.003872,0.000538,0.003872,0.000510
2,0.004960,0.003883,0.004960,0.003228,0.004960,0.002341,0.004960,0.000686,0.004960,0.000650
3,0.006336,0.004950,0.006336,0.004107,0.006336,0.002975,0.006336,0.000872,0.006336,0.000827
4,0.008074,0.006294,0.008074,0.005213,0.008074,0.003771,0.008074,0.001106,0.008074,0.001050
5,0.010263,0.007984,0.010263,0.006600,0.010263,0.004770,0.010263,0.001399,0.010263,0.001329
6,0.013013,0.010103,0.013013,0.008337,0.013013,0.006020,0.013013,0.001765,0.013013,0.001678
7,0.016457,0.012754,0.016457,0.010507,0.016457,0.007579,0.016457,0.002223,0.016457,0.002114
8,0.020761,0.016061,0.020761,0.013210,0.020761,0.009520,0.020761,0.002792,0.020761,0.002656
9,0.026125,0.020175,0.026125,0.016569,0.026125,0.011930,0.026125,0.003500,0.026125,0.003331


In [ ]:
# Import necessary packages
import numpy as np
from estimate_damages import *
from IPython.core.display import display, HTML
import plotly.io as pio
import matplotlib.pyplot as plt
from scipy.stats import norm
import pandas as pd

try:   
    import plotly.graph_objs as go
    from plotly.tools import make_subplots
    from plotly.offline import init_notebook_mode, iplot
#     import plotly.express as px
except ImportError:
    print("Installing plotly. This may take a while.")
    from pip._internal import main as pipmain
    pipmain(['install', 'plotly'])
    import plotly.graph_objs as go
    from plotly.subplots import make_subplots
    from plotly.offline import init_notebook_mode, iplot
#     import plotly.express as px

init_notebook_mode()
quad_rule = 'Legendre_5sd'


In [170]:
RE_2_func(hists2[tm,:])

array([3.35807153e+02, 5.11469256e+03, 9.40577334e+02, 6.38086147e-03,
       8.84126785e-03])

In [ ]:
for_check = loadmat("SCC2.mat")
np.max(abs(SCC2_FK_tilt - for_check['SCC2_FK_tilt'].reshape(SCC2_FK_tilt.shape)) / SCC2_FK_tilt)

In [17]:
(np.log(hists2[tm,1,path]),hists2[tm, 3, path], np.log(hists2[tm,2,path]))

(6.544849785292769, 4.2550764047116534e-05, 5.66988092298052)

In [147]:
np.c_[SCC2_FK_base, for_check['SCC2_FK_base']]

array([[ 32.0984231 ,  31.52482893],
       [ 32.27350894,  31.69852201],
       [ 32.44870059,  31.87331216],
       [ 32.62399764,  32.04920652],
       [ 32.7993997 ,  32.22621223],
       [ 32.9749064 ,  32.40433647],
       [ 33.15051733,  32.5835864 ],
       [ 33.32623213,  32.76396922],
       [ 33.50728883,  32.94549215],
       [ 33.69151055,  33.12816305],
       [ 33.87590079,  33.31198992],
       [ 34.06045897,  33.49698058],
       [ 34.24518449,  33.68314286],
       [ 34.43007676,  33.87048465],
       [ 34.61513521,  34.05901382],
       [ 34.80035924,  34.24873829],
       [ 34.98574828,  34.43966601],
       [ 35.17130175,  34.63180493],
       [ 35.35701907,  34.82516305],
       [ 35.54289967,  35.0197484 ],
       [ 35.72894298,  35.21556901],
       [ 35.91514843,  35.41263297],
       [ 36.10151545,  35.61094837],
       [ 36.28804347,  35.81052337],
       [ 36.47473195,  36.01136613],
       [ 36.66158031,  36.21348485],
       [ 36.84858801,  36.41688776],
 

In [148]:
np.max(abs(for_check['external_v0_worst'] - v0_worst))

1.030989990866122e-05

In [202]:
σ

SyntaxError: unexpected character after line continuation character (<ipython-input-202-c100df21a0ee>, line 1)

In [77]:
np.max(for_check['nordhaus_model_drift'] - nordhaus_model_drift)

NameError: name 'for_check' is not defined

In [82]:
e_func(hist2[0,:])

0.014912017469889226

In [10]:
pi_tilde_2_func(initial_val)

1.2052765670584387

In [126]:
base_drift_func = scipy.interpolate.RegularGridInterpolator(gridpoints, base_model_drift)
base_drift_func = lambda x: base_drift_func([np.log(x[0]), x[2], np.log(x[1])])

In [7]:
for_check = loadmat('simulation.mat')

C:\Softwares\Anaconda\lib\site-packages\scipy\io\matlab\mio.py:208: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


In [131]:

# 

TypeError: 'int' object is not subscriptable

In [120]:
x = (1,2,3)
base_drift_func((1,2,3))

TypeError: <lambda>() takes 1 positional argument but 3 were given

In [113]:
pts[0]

array([1, 2, 3])

In [124]:

def ffff(x,y,z):
    return 2 * x**3 + 3 * y**2 - z
x = np.linspace(1, 4, 11)
y = np.linspace(4, 7, 22)
z = np.linspace(7, 9, 33)
data = ffff(*np.meshgrid(x, y, z, indexing='ij', sparse=True))

my_interpolating_function = scipy.interpolate.RegularGridInterpolator((x, y, z), data)
pts = np.array([[2.1, 6.2, 8.3], [3.3, 5.2, 7.1]])
my_interpolating_function(pts)


array([125.80469388, 146.30069388])

In [125]:
my_interpolating_function([2.1,6.2,8.4])

array([125.70469388])

In [46]:
b

0.004196590630811646

In [47]:
n

30

In [28]:
np.diff(x)

array([0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3])

In [29]:
x

array([1. , 1.3, 1.6, 1.9, 2.2, 2.5, 2.8, 3.1, 3.4, 3.7, 4. ])